# Crashes Near Point

Find the crash incident_id for a specific crash given its location and time. 

In [1]:
import folium
import datetime
import pandas as pd
from folium.plugins import HeatMap

from crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()
# df = cda.crash_dataframe(verbose=True)

scooter rider injured in hit and run 

https://www.denver7.com/news/local-news/denver-woman-recovering-from-serious-injuries-after-hit-and-run-while-riding-scooter

incident_id = 2023182255

In [3]:
# University Blvd and E 3rd Ave (2022,12,14) incident_id = 2022639830
# lat_lon = [39.720934, -104.959436]

# Speer Blvd and W 11th Ave
# lat_lon = [39.73378086237446, -104.99382870794332]

# Sheridan and W 38th Ave
# lat_lon = [39.76932995840248, -105.05322879106377]

# 18th St and Wazee - 2023-01-07
# lat_lon = [39.753095972145644, -104.99765515593928]

# 17th Ave and Humboldt
# lat_lon = [39.74325582993406, -104.96969463423684]

# Evans and Holly
# lat_lon = [39.678456222120396, -104.92219658639499]

# Perry and 32nd
# lat_lon = [39.76207714280117, -105.03931855940714]

# W 23rd Ave and Julian
# lat_lon = [39.751187102730654, -105.03155988650708]

# 35th St and Welton
# lat_lon = [39.76762137425839, -104.97535848218475]

# 1776 Race St
# lat_lon = [39.744484814811436, -104.96299454652576]

# 730 S Canosa Ct
# lat_lon = [39.70332702509137, -105.01894998129806]

# Sherman St house
# lat_lon = [39.721383, -104.984736]

# Broadway and Colfax - scooter rider injured in hit and run
# lat_lon = [39.74012825562107, -104.98743057571703]

lat_lon = cda.geocode_location('W 14th Ave & Federal Blvd, Denver, CO')

In [4]:
# crash occurred within this distance of the given latitude and longitude
radius_miles = 0.25

query = f"""
select
*
, reported_date at time zone 'America/Denver' as reported_date_local
, ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
) as distance_feet

from crashes

where geo_lon is not null
and ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
    ) < {radius_miles * 5280}

order by reported_date
"""
# and extract(year from reported_date) >= 2022

nearby = pd.read_sql(query, cda.conn)

In [5]:
nearby.groupby(nearby.reported_date.dt.year).agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date,,,,,
2013,90,1,1,2,0
2014,98,1,2,6,2
2015,94,1,2,9,2
2016,74,0,3,3,0
2017,77,1,3,9,0
2018,75,0,2,6,0
2019,73,1,2,3,0
2020,42,0,0,2,0
2021,61,0,6,4,2


In [6]:
nearby['all_time'] = 'all time'
nearby.groupby('all_time').agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
all_time,,,,,
all time,760,9,24,52,7


In [7]:
display_cols = [
    'reported_date_local'
    , 'top_traffic_accident_offense'
    , 'incident_address_corrected'
    , 'bicycle_ind'
    , 'pedestrian_ind'
    , 'sbi'
    , 'fatality'
    , 'distance_feet'
    , 'incident_id'
]
# nearby.to_clipboard()
# nearby[display_cols]

In [8]:
nearby[display_cols] #.tail(5)

,reported_date_local,top_traffic_accident_offense,incident_address_corrected,bicycle_ind,pedestrian_ind,sbi,fatality,distance_feet,incident_id
0,2013-01-12 17:40:00,TRAF - ACCIDENT,W COLFAX AVE / N ELIOT ST,0.0,0.0,False,False,1015.757391,201318926
1,2013-01-17 09:31:00,TRAF - ACCIDENT,1400 BLOCK N FEDERAL BLVD/ W 14 TH AVE,0.0,0.0,False,False,332.958368,201326264
2,2013-01-31 18:41:00,TRAF - ACCIDENT,1200 N FEDERAL BLVD,0.0,0.0,False,False,1083.384434,201350496
3,2013-02-01 15:54:00,TRAF - ACCIDENT,W HOLDEN PL / N FEDERAL BLVD,0.0,1.0,False,False,850.061024,201352143
4,2013-02-06 10:37:00,TRAF - ACCIDENT,W 14TH AVE / N FEDERAL BLVD,0.0,0.0,False,False,40.909350,201359833
...,...,...,...,...,...,...,...,...,...
755,2023-03-29 11:57:00,TRAF - ACCIDENT - HIT & RUN,3200 BLOCK W COLFAX AVE,0.0,0.0,False,False,1206.121389,2023161548
756,2023-03-31 12:57:00,TRAF - ACCIDENT,1400 BLOCK N FEDERAL BLVD,0.0,0.0,False,False,331.811781,2023165694
757,2023-04-02 08:34:00,TRAF - ACCIDENT - HIT & RUN,3010 W 14TH AVE,0.0,0.0,False,False,768.023755,2023169399
758,2023-04-16 13:52:00,TRAF - ACCIDENT,N FEDERAL BLVD / W 14TH AVE,0.0,0.0,False,False,40.909350,2023196515


In [9]:
this_map = folium.Map(prefer_canvas=True)
HeatMap(nearby[['geo_lat', 'geo_lon']]).add_to(this_map)
this_map.fit_bounds(this_map.get_bounds())
this_map

In [10]:
cda.conn.dispose()